In [76]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model
from qiskit_optimization.converters import QuadraticProgramToQubo
from typing import List
from itertools import permutations
# useful additional packages
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
import networkx as nx
from qiskit import Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.problems import QuadraticProgram

In [81]:
class FSPasTSP:
     """Quantum Optimization for the FSP by mapping it to FSP"""
     def __init__(self,numberMachine : int,procTime:List[List[int]],numberJobs : int)-> None :
        """
        Args :
        timeSpan : the makespan value
        numberMachine : machine number
        numverJobs : job's number
        """
        self.numberMachine = numberMachine
        self.numberJobs = numberJobs
        self.procTime = procTime

     def convert_FSP_WH(self) -> dict[tuple[int, int], int]:
         """
             We are using Widemar & Hertz approach
             duv=p1,u + \sum_2^m (m-i)|pi,u - pi-1,v| + pmv
         """
         TspInstance={
             (u,v) :
             self.procTime[1][u]
             +self.procTime[self.numberMachine-1][v]
             +sum([(self.numberMachine-i)*abs(self.procTime[i][u]-self.procTime[i-1][v]) for i in range(1,self.numberMachine)])
             for u in range(self.numberJobs)
             for v in range(self.numberJobs)
             if u!=v
             }
         return TspInstance

     def convert_FSP_SS6(self) -> dict[tuple[int, int], int]:
         TspInstance={
             (u,v):
             sum([max(self.procTime[i][u]-self.procTime[i-1][v],0)+2*abs(min(self.procTime[i][u]-self.procTime[i-1][v],0)) for i in range(1,self.numberMachine)])
             for u in range(self.numberJobs)
             for v in range(self.numberJobs)
             if u!=v
             }
         return TspInstance

     def convert_FSP_Gupta(self) -> dict[tuple[int, int], int]:
         def CT_j (u,v,j) -> int :
             ct = 0
             if j == 0 : return ct
             ct = max( CT_j(u,v,j-1) , sum([self.procTime[i][v] for i in range(j)]) )
             return ct

         TspInstance = {
            (u,v) :
            CT_j(u,v,self.numberMachine) - sum([self.procTime[i][u] for i in range(self.numberMachine)])
            for u in range(self.numberJobs)
            for v in range(self.numberJobs)
            if u != v
         }
         return TspInstance

     def convert_FSP_Moccelin(self) -> dict[tuple[int, int], int]:
          def UBX(u,v,m) -> int:
              ubx = 0
              if m == 0 : return ubx
              ubx = max (0, ubx(u,v,m-1) + self.procTime[m-1][u] - self.procTime[m][u])
              return ubx
          TspInstance = {
            (u,v) :
            UBX(u,v,self.numberMachine) - sum([self.procTime[i][u] for i in range(self.numberMachine)])
            for u in range(self.numberJobs)
            for v in range(self.numberJobs)
            if u != v
         }
          return TspInstance
     
     def convert_FSP_SS1(self) -> dict[tuple[int,int],int]:
         TspInstance = {
            (u,v) :
            sum([abs(self.procTime[i][u]-self.procTime[i-1][v]) for i in range(1,self.numberMachine)])
            for u in range(self.numberJobs)
            for v in range(self.numberJobs)
            if u != v
         }
         return TspInstance
     
     def quadratic_program(self)->QuadraticProgram:
         TspIns=self.convert_FSP_WH()
         mdl=Model(name="TSP")
         n=self.numberJobs
         x = {(i, k): mdl.binary_var(name=f"x_{i}_{k}") for i in range(n) for k in range(n)}
         cost_fun=mdl.sum(
             TspIns[(u,v)]* x[(u, k)] * x[(v, (k + 1) % n)]
             for u in range(n)
             for v in range(n)
             for k in range(n)
             if u != v
         )
         mdl.minimize(cost_fun)
         for u in range(n):
            mdl.add_constraint(mdl.sum(x[(u, k)] for k in range(n)) == 1)
         for k in range(n):
            mdl.add_constraint(mdl.sum(x[(u, k)] for u in range(n)) == 1)
         op = from_docplex_mp(mdl)
         return op
     
     def draw(self):
          instance = self.convert_FSP_WH()
          edges = [(u,v) for u,v in instance.keys()]
          graph = nx.DiGraph()
          graph.add_nodes_from([i for i in range(self.numberJobs)])
          graph.add_weighted_edges_from(
              [(u,v,instance[(u,v)])
              for u,v in edges]
              )
          pos = nx.spring_layout(graph, seed=6)
          weight_labels = nx.get_edge_attributes(graph,'weight')
          nx.draw(graph,pos,font_color = 'white', node_shape = 's', with_labels = True,)
          nx.draw_networkx_edge_labels(graph,pos,edge_labels=weight_labels)
          return graph

     def QUBO(self)->QuadraticProgram :
         conv = QuadraticProgramToQubo()
         return conv.convert(self.quadratic_program())

     def brute_force(self) :
         tsp = self.convert_FSP_WH()
         a =list(permutations(range(1,self.numberJobs)))
         last_best_distance = 1e10
         for i in a :
                distance = 0
                pre_j = 0
                for j in i:
                    if (pre_j != 0) : distance = distance + tsp[( pre_j,j)]
                    pre_j = j
                distance = distance + tsp[(0,pre_j)]
                order = (0,) + i
                if distance < last_best_distance:
                    best_order = order
                    last_best_distance = distance
         return last_best_distance, best_order

     #def compare_Distance():  
class QUBOSolver():
    def __init__(self, problem : str, method : int , quantumInstance :  QuantumInstance , data : str)->None:
        """
         Different approach to solve QUBO
        """
        self.problem = problem
        self.method = method
        self.quantumInstance = quantumInstance
        self.data = data

    def read_Data(self) :
        Instance,l,M = {},[],[]
        i,p,c = 0,0,''
        file = open('instances.txt',"r")
        ins = file.read().split()
        while i in range(len(ins)):
            n = int(ins[i][0])
            m = int(ins[i][2])
            for k in range(i+1,i+m+1):
                for j in range(len(ins[k])) :
                    if ins[k][j] == ',' : 
                        l.append(c)
                        c=''
                    else : c += ins[k][j] 
                M.append(l)
                l= []
            Instance[p]=n,m,M
            M = []
            i = i + m +1 
            p +=1
        return Instance

    #def VQE():
    
    #def MES(): #exact method

    #def valid_Solution() : #exact method

    #def QAOA () :

    #def save_Sol():
            

        

fsp = FSPasTSP(2,[[3,5,4],[5,4,5]],3)
distance,order = fsp.brute_force()
qb = QUBOSolver(None,None,None,data = 'instances.txt')
instances = qb.read_Data()
instances



{0: (3, 2, [['4', '5', '6'], ['5', '8', '9']]),
 1: (3, 3, [['54', '83', '15'], ['79', '3', '11'], ['16', '89', '49']]),
 2: (3,
  4,
  [['26', '38', '27'],
   ['59', '62', '44'],
   ['78', '90', '64'],
   ['69', '30', '61']]),
 3: (3,
  5,
  [['26', '38', '27'],
   ['59', '62', '44'],
   ['78', '90', '64'],
   ['88', '54', '47'],
   ['69', '30', '61']]),
 4: (4, 2, [['55', '54', '63', '14'], ['13', '11', '5', '11']]),
 5: (4,
  3,
  [['3', '4', '5', '6'], ['10', '45', '2', '1'], ['6', '8', '9', '4']]),
 6: (4,
  4,
  [['54', '71', '78', '9'],
   ['86', '25', '39', '76'],
   ['10', '79', '93', '2'],
   ['71', '32', '48', '42']]),
 7: (4,
  5,
  [['53', '19', '99', '62'],
   ['93', '79', '88', '77'],
   ['90', '92', '35', '13'],
   ['65', '97', '27', '25'],
   ['64', '38', '44', '46']]),
 8: (5, 2, [['71', '27', '55', '90', '11'], ['13', '11', '73', '43', '27']]),
 9: (5,
  3,
  [['15', '64', '64', '48', '9'],
   ['28', '4', '43', '93', '1'],
   ['77', '36', '57', '15', '81']])}

In [ ]:
Instance = {}
file = open('instances.txt',"r")
ins = file.read().split()
l,M = [],[]
i,p,c = 0,0,''
while i in range(len(ins)):
    n = int(ins[i][0])
    m = int(ins[i][2])
    for k in range(i+1,i+m+1):
        for j in range(len(ins[k])) :
            if ins[k][j] == ',' : 
                l.append(c)
                c=''
            else : c += ins[k][j] 
        M.append(l)
        l= []
    Instance[p]=n,m,M
    M = []
    i = i + m +1 
    p +=1
Instance

{0: (3, 2, [['4', '5', '6'], ['5', '8', '9']]),
 1: (3, 3, [['54', '83', '15'], ['79', '3', '11'], ['16', '89', '49']]),
 2: (3,
  4,
  [['26', '38', '27'],
   ['59', '62', '44'],
   ['78', '90', '64'],
   ['69', '30', '61']]),
 3: (3,
  5,
  [['26', '38', '27'],
   ['59', '62', '44'],
   ['78', '90', '64'],
   ['88', '54', '47'],
   ['69', '30', '61']]),
 4: (4, 2, [['55', '54', '63', '14'], ['13', '11', '5', '11']]),
 5: (4,
  3,
  [['3', '4', '5', '6'], ['10', '45', '2', '1'], ['6', '8', '9', '4']]),
 6: (4,
  4,
  [['54', '71', '78', '9'],
   ['86', '25', '39', '76'],
   ['10', '79', '93', '2'],
   ['71', '32', '48', '42']]),
 7: (4,
  5,
  [['53', '19', '99', '62'],
   ['93', '79', '88', '77'],
   ['90', '92', '35', '13'],
   ['65', '97', '27', '25'],
   ['64', '38', '44', '46']]),
 8: (5, 2, [['71', '27', '55', '90', '11'], ['13', '11', '73', '43', '27']]),
 9: (5,
  3,
  [['15', '64', '64', '48', '9'],
   ['28', '4', '43', '93', '1'],
   ['77', '36', '57', '15', '81']])}